<a href="https://colab.research.google.com/github/mshojaei77/Awesome-Fine-tuning/blob/main/gemma_2_axolotl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate peft transformers trl torch bitsandbytes
!pip install -q ninja packaging datasets
!pip install -q flash-attn --no-build-isolation

In [2]:
!git clone https://github.com/OpenAccess-AI-Collective/axolotl.git
%cd axolotl

Cloning into 'axolotl'...
remote: Enumerating objects: 14582, done.
remote: Counting objects: 100% (1312/1312), done.
remote: Compressing objects: 100% (481/481), done.
remote: Total 14582 (delta 814), reused 1145 (delta 691), pack-reused 13270
Receiving objects: 100% (14582/14582), 5.68 MiB | 25.75 MiB/s, done.
Resolving deltas: 100% (9381/9381), done.
/content/axolotl


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset

# Replace 'your_dataset' with the actual dataset name
dataset = load_dataset("mshojaei77/Dolly_Alpaca_Lmsys")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

In [ ]:
%%writefile config.yaml
# Configuration for fine-tuning the Gemma 2-9B model on a T4 GPU

base_model: google/gemma-2-9b-it
model_type: GemmaForCausalLM
tokenizer_type: GemmaTokenizer

# Use GemmaConfig to initialize model parameters
model_config:
  class: GemmaConfig
  parameters: {}

datasets:
  - path: mshojaei77/Dolly_Alpaca_Lmsys
    type: alpaca

dataset_prepared_path: last_run_prepared
val_set_size: 0.05
output_dir: ./results

sequence_len: 1024  # Reduced to fit T4 GPU memory constraints
sample_packing: true
pad_to_sequence_len: true

adapter: lora
lora_model_dir: ./lora_model_dir  # Specify the directory for LoRA model
lora_r: 8  # Reduced for T4 GPU compatibility
lora_alpha: 16  # Adjusted for memory efficiency
lora_dropout: 0.1  # Dropout for regularization
lora_target_modules:
  - q_proj
  - v_proj

load_in_8bit: true  # Enable 8-bit loading for memory efficiency
load_in_4bit: false

train_batch_size: 1  # Reduced to fit T4 GPU memory constraints
micro_batch_size: 1
num_epochs: 3  # Adjusted for quicker runs on Colab
optimizer: adamw_bnb_8bit
lr_scheduler: cosine
learning_rate: 0.0002  # Slightly increased to speed up training
warmup_steps: 100  # Reduced for quicker runs
eval_steps: 100  # Less frequent evaluation to save time
save_steps: 200  # Save less frequently to save disk space
save_total_limit: 2  # Fewer checkpoints to save disk space
logging_steps: 20
weight_decay: 0.01

gradient_accumulation_steps: 1  # Simplified for Colab
gradient_checkpointing: true
gradient_checkpointing_kwargs:
  use_reentrant: false

bf16: true  # Use bf16 for faster training with T4 compatibility
fp16: false
tf32: false  # T4 GPUs do not support TF32

early_stopping_patience: 3  # Adjusted for quicker runs
resume_from_checkpoint: false  # Start training from scratch each time
local_rank: 0  # Typically set automatically by the distributed training framework


In [ ]:
!python axolotl/src/axolotl/cli/train.py --config_file=config.yaml

In [ ]:
model.save_pretrained("./fine_tuned_gemma2")
tokenizer.save_pretrained("./fine_tuned_gemma2")
model.push_to_hub("mshojaei77/fine_tuned_gemma2")
tokenizer.push_to_hub("mshojaei77/fine_tuned_gemma2")

In [ ]:
!python axolotl/src/axolotl/cli/convert.py --model_name_or_path="./fine_tuned_gemma2" --output_dir="./gguf"
from huggingface_hub import hf_hub_download
from huggingface_hub import HfApi
api = HfApi()
api.upload_folder(
    folder_path="./gguf",
    repo_id="mshojaei77/fine_tuned_gemma2",
    repo_type="model",
)


In [ ]:
from transformers import pipeline

fine_tuned_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_gemma2")
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_gemma2")

text_generator = pipeline("text-generation", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)
print(text_generator("Once upon a time,")[0]['generated_text'])